In [23]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install japanize_matplotlib bottleneck tslearn

In [24]:
#作った関数
import MasterResearchFunction as mr
# 基本ライブラリ
import os, re, csv, math, statistics
from datetime import datetime, timedelta
from decimal import Decimal

# 数値計算とデータ処理
import numpy as np
import pandas as pd
import bottleneck as bn

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Conv1D, MaxPooling1D, Flatten

# プロットと可視化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib

# その他のライブラリ
from scipy import signal, stats
from scipy.signal import savgol_filter
from tslearn.metrics import dtw_path

In [ ]:
#Apple Watchのモーションデータの読み込み
# a_yuuma_motion_data = mr.process_apple_watch_csv("datasets/yuuma/20240604/yuuma_motion.csv")
# b_sakamoto_motion_data = mr.process_apple_watch_csv("datasets/sakamoto/20240604/sakamoto_motion.csv")
# c_watabe_motion_data = mr.process_apple_watch_csv("datasets/watabe/20240605/watabe_motion.csv")
# d_nakazawa_motion_data = mr.process_apple_watch_csv("datasets/nakazawa/20240606/nakazawa_motion.csv")
# e_okede_motion_data = mr.process_apple_watch_csv("datasets/okeda/20240607/okeda_motion.csv")
a_yuuma_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_motion.csv")
b_sakamoto_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_motion.csv")
c_watabe_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_motion.csv")
d_nakazawa_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_motion.csv")
e_okeda_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_motion.csv")

In [ ]:
#Tobiiのモーションデータの読み込み
# a_yuuma_eye_data = mr.process_tobii_csv("datasets/yuuma/20240604/yuuma_eye.csv")
# b_sakamoto_eye_data = mr.process_tobii_csv("datasets/sakamoto/20240604/sakamoto_eye.csv")
# c_watabe_eye_data = mr.process_tobii_csv("datasets/watabe/20240605/watabe_eye.csv")
# d_nakazawa_eye_data = mr.process_tobii_csv("datasets/nakazawa/20240606/nakazawa_eye.csv")
# e_okeda_eye_data = mr.process_tobii_csv("datasets/okeda/20240607/okeda_eye.csv")
a_yuuma_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_eye.csv")
b_sakamoto_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_eye.csv")
c_watabe_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_eye.csv")
d_nakazawa_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_eye.csv")
e_okeda_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_eye.csv")

In [ ]:
#a_yuumaくんの教師データ読み込み
# a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/check")
# a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/circle")
# a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/cross")
# a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/tri")
a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/check")
a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/circle")
a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/cross")
a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/tri")

In [ ]:
#b_sakamotoくんの教師データ読み込み
# b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/check")
# b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/circle")
# b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/cross")
# b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/tri")
b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/check")
b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/circle")
b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/cross")
b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/tri")

In [ ]:
#c_watabeくんの教師データ読み込み
# c_watabe_check = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/check")
# c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/circle")
# c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/cross")
# c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/tri")
c_watabe_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/check")
c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/circle")
c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/cross")
c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/tri")

In [ ]:
#d_nakazawaくんの教師データ読み込み
# d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/check")
# d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/circle")
# d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/cross")
# d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/tri")
d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/check")
d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/circle")
d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/cross")
d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/tri")

In [ ]:
#e_okedaくんの教師データ読み込み
# e_okeda_check = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/check")
# e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/circle")
# e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/cross")
# e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/tri")
e_okeda_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/check")
e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/circle")
e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/cross")
e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/tri")

In [25]:
Th = 4
# segx, segy, segz = [], [], []

In [26]:
Type = 'acc'
#Type = 'gyro'

In [ ]:
a_yuuma_circle_segx, a_yuuma_circle_segy, a_yuuma_circle_segz = mr.three_axis_spring_ogawa(a_yuuma_motion_data, a_yuuma_circle, Th, Type)
b_sakamoto_circle_segx, b_sakamoto_circle_segy, b_sakamoto_circle_segz = mr.three_axis_spring_ogawa(b_sakamoto_motion_data, b_sakamoto_circle, Th, Type)
c_watabe_circle_segx, c_watabe_circle_segy, c_watabe_circle_segz = mr.three_axis_spring_ogawa(c_watabe_motion_data, c_watabe_circle, Th, Type)
d_nakazawa_circle_segx, d_nakazawa_circle_segy, d_nakazawa_circle_segz = mr.three_axis_spring_ogawa(d_nakazawa_motion_data, d_nakazawa_circle, Th, Type)

ストリーミング出力は最後の 5000 行に切り捨てられました。
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
***

In [ ]:
import pickle

# 保存する変数を辞書にまとめる
data = {
    'a_yuuma_circle_segx': a_yuuma_circle_segx,
    'a_yuuma_circle_segy': a_yuuma_circle_segy,
    'a_yuuma_circle_segz': a_yuuma_circle_segz,
    'b_sakamoto_circle_segx': b_sakamoto_circle_segx,
    'b_sakamoto_circle_segy': b_sakamoto_circle_segy,
    'b_sakamoto_circle_segz': b_sakamoto_circle_segz,
    'c_watabe_circle_segx': c_watabe_circle_segx,
    'c_watabe_circle_segy': c_watabe_circle_segy,
    'c_watabe_circle_segz': c_watabe_circle_segz,
    'd_nakazawa_circle_segx': d_nakazawa_circle_segx,
    'd_nakazawa_circle_segy': d_nakazawa_circle_segy,
    'd_nakazawa_circle_segz': d_nakazawa_circle_segz
}

# 変数をpickleファイルに保存
with open('circle_segments4.pkl', 'wb') as f:
    pickle.dump(data, f)

# pickleファイルをダウンロードするためのコード（Colabの場合）
from google.colab import files
files.download('circle_segments4.pkl')

In [ ]:
 Th = 5

In [ ]:
a_yuuma_cross_segx, a_yuuma_cross_segy, a_yuuma_cross_segz = mr.three_axis_spring_ogawa(a_yuuma_motion_data, a_yuuma_cross, Th, Type)
b_sakamoto_cross_segx, b_sakamoto_cross_segy, b_sakamoto_cross_segz = mr.three_axis_spring_ogawa(b_sakamoto_motion_data, b_sakamoto_cross, Th, Type)
c_watabe_cross_segx, c_watabe_cross_segy, c_watabe_cross_segz = mr.three_axis_spring_ogawa(c_watabe_motion_data, c_watabe_cross, Th, Type)
d_nakazawa_cross_segx, d_nakazawa_cross_segy, d_nakazawa_cross_segz = mr.three_axis_spring_ogawa(d_nakazawa_motion_data, d_nakazawa_cross, Th, Type)

In [ ]:
# 保存する変数を辞書にまとめる
data = {
    'a_yuuma_cross_segx': a_yuuma_cross_segx,
    'a_yuuma_cross_segy': a_yuuma_cross_segy,
    'a_yuuma_cross_segz': a_yuuma_cross_segz,
    'b_sakamoto_cross_segx': b_sakamoto_cross_segx,
    'b_sakamoto_cross_segy': b_sakamoto_cross_segy,
    'b_sakamoto_cross_segz': b_sakamoto_cross_segz,
    'c_watabe_cross_segx': c_watabe_cross_segx,
    'c_watabe_cross_segy': c_watabe_cross_segy,
    'c_watabe_cross_segz': c_watabe_cross_segz,
    'd_nakazawa_cross_segx': d_nakazawa_cross_segx,
    'd_nakazawa_cross_segy': d_nakazawa_cross_segy,
    'd_nakazawa_cross_segz': d_nakazawa_cross_segz
}

# 変数をpickleファイルに保存
with open('cross_segments5.pkl', 'wb') as f:
    pickle.dump(data, f)

# pickleファイルをダウンロードするためのコード（Colabの場合）
from google.colab import files
files.download('cross_segments5.pkl')

In [ ]:
a_yuuma_tri_segx, a_yuuma_tri_segy, a_yuuma_tri_segz = mr.three_axis_spring_ogawa(a_yuuma_motion_data, a_yuuma_tri, Th, Type)
b_sakamoto_tri_segx, b_sakamoto_tri_segy, b_sakamoto_tri_segz = mr.three_axis_spring_ogawa(b_sakamoto_motion_data, b_sakamoto_tri, Th, Type)
c_watabe_tri_segx, c_watabe_tri_segy, c_watabe_tri_segz = mr.three_axis_spring_ogawa(c_watabe_motion_data, c_watabe_tri, Th, Type)
d_nakazawa_tri_segx, d_nakazawa_tri_segy, d_nakazawa_tri_segz = mr.three_axis_spring_ogawa(d_nakazawa_motion_data, d_nakazawa_tri, Th, Type)

In [ ]:
# 保存する変数を辞書にまとめる
data = {
    'a_yuuma_tri_segx': a_yuuma_tri_segx,
    'a_yuuma_tri_segy': a_yuuma_tri_segy,
    'a_yuuma_tri_segz': a_yuuma_tri_segz,
    'b_sakamoto_tri_segx': b_sakamoto_tri_segx,
    'b_sakamoto_tri_segy': b_sakamoto_tri_segy,
    'b_sakamoto_tri_segz': b_sakamoto_tri_segz,
    'c_watabe_tri_segx': c_watabe_tri_segx,
    'c_watabe_tri_segy': c_watabe_tri_segy,
    'c_watabe_tri_segz': c_watabe_tri_segz,
    'd_nakazawa_tri_segx': d_nakazawa_tri_segx,
    'd_nakazawa_tri_segy': d_nakazawa_tri_segy,
    'd_nakazawa_tri_segz': d_nakazawa_tri_segz
}

# 変数をpickleファイルに保存
with open('tri_segments5.pkl', 'wb') as f:
    pickle.dump(data, f)

# pickleファイルをダウンロードするためのコード（Colabの場合）
from google.colab import files
files.download('tri_segments5.pkl')

In [ ]:
a_yuuma_check_segx, a_yuuma_check_segy, a_yuuma_check_segz = mr.three_axis_spring_ogawa(a_yuuma_motion_data, a_yuuma_check, Th, Type)
b_sakamoto_check_segx, b_sakamoto_check_segy, b_sakamoto_check_segz = mr.three_axis_spring_ogawa(b_sakamoto_motion_data, b_sakamoto_check, Th, Type)
c_watabe_check_segx, c_watabe_check_segy, c_watabe_check_segz = mr.three_axis_spring_ogawa(c_watabe_motion_data, c_watabe_check, Th, Type)
d_nakazawa_check_segx, d_nakazawa_check_segy, d_nakazawa_check_segz = mr.three_axis_spring_ogawa(d_nakazawa_motion_data, d_nakazawa_check, Th, Type)

In [ ]:
# 保存する変数を辞書にまとめる
data = {
    'a_yuuma_check_segx': a_yuuma_check_segx,
    'a_yuuma_check_segy': a_yuuma_check_segy,
    'a_yuuma_check_segz': a_yuuma_check_segz,
    'b_sakamoto_check_segx': b_sakamoto_check_segx,
    'b_sakamoto_check_segy': b_sakamoto_check_segy,
    'b_sakamoto_check_segz': b_sakamoto_check_segz,
    'c_watabe_check_segx': c_watabe_check_segx,
    'c_watabe_check_segy': c_watabe_check_segy,
    'c_watabe_check_segz': c_watabe_check_segz,
    'd_nakazawa_check_segx': d_nakazawa_check_segx,
    'd_nakazawa_check_segy': d_nakazawa_check_segy,
    'd_nakazawa_check_segz': d_nakazawa_check_segz
}

# 変数をpickleファイルに保存
with open('check_segments5.pkl', 'wb') as f:
    pickle.dump(data, f)

# pickleファイルをダウンロードするためのコード（Colabの場合）
from google.colab import files
files.download('check_segments5.pkl')

In [ ]:
a_yuuma_circle_overlap = mr.overlap_from_three_segments(a_yuuma_circle_segx, a_yuuma_circle_segy, a_yuuma_circle_segz)
b_sakamoto_circle_overlap = mr.overlap_from_three_segments(b_sakamoto_circle_segx, b_sakamoto_circle_segy, b_sakamoto_circle_segz)
c_watabe_circle_overlap = mr.overlap_from_three_segments(c_watabe_circle_segx, c_watabe_circle_segy, c_watabe_circle_segz)
d_nakazawa_circle_overlap = mr.overlap_from_three_segments(d_nakazawa_circle_segx, d_nakazawa_circle_segy, d_nakazawa_circle_segz)

In [ ]:
# 経過時間のフィルタリングを適用
a_yuuma_filtered_overlap = mr.filter_overlap_by_elapsed_time(a_yuuma_circle_overlap, a_yuuma_motion_data, min_time=2, max_time=5)
b_sakamoto_filtered_overlap = mr.filter_overlap_by_elapsed_time(b_sakamoto_circle_overlap, b_sakamoto_motion_data, min_time=2, max_time=5)
c_watabe_filtered_overlap = mr.filter_overlap_by_elapsed_time(c_watabe_circle_overlap, c_watabe_motion_data, min_time=2, max_time=5)
d_nakazawa_filtered_overlap = mr.filter_overlap_by_elapsed_time(d_nakazawa_circle_overlap, d_nakazawa_motion_data, min_time=2, max_time=5)

In [ ]:
# オーバーラップを統合
a_yuuma_combine_overlap = mr.combine_all_overlaps(a_yuuma_filtered_overlap)
b_sakamoto_combine_overlap = mr.combine_all_overlaps(b_sakamoto_filtered_overlap)
c_watabe_combine_overlap = mr.combine_all_overlaps(c_watabe_filtered_overlap)
d_nakazawa_combine_overlap = mr.combine_all_overlaps(d_nakazawa_filtered_overlap)

In [ ]:
# 統合された組からタイムスタンプの組を抽出する
a_yuuma_results =  mr.extract_timestamp_from_overlap(a_yuuma_motion_data, a_yuuma_combine_overlap)
b_sakamoto_results =  mr.extract_timestamp_from_overlap(b_sakamoto_motion_data, b_sakamoto_combine_overlap)
c_watabe_results =  mr.extract_timestamp_from_overlap(c_watabe_motion_data, c_watabe_combine_overlap)
d_nakazawa_results =  mr.extract_timestamp_from_overlap(d_nakazawa_motion_data, d_nakazawa_combine_overlap)

In [ ]:
# 各組の時間の長さが2秒以下と5秒以上のものを弾く
a_yuuma_filtered_results = mr.filter_segments_by_time_length(a_yuuma_results, 2, 5)
b_sakamoto_filtered_results = mr.filter_segments_by_time_length(b_sakamoto_results, 2, 5)
c_watabe_filtered_results = mr.filter_segments_by_time_length(c_watabe_results, 2, 5)
d_nakazawa_filtered_results = mr.filter_segments_by_time_length(d_nakazawa_results, 2, 5)

In [ ]:
# タイムスタンプの範囲内のデータを抽出
a_yuuma_extracted_eye_data = mr.extract_eye_data_within_intervals(a_yuuma_filtered_results, a_yuuma_eye_data)
b_sakamoto_extracted_eye_data = mr.extract_eye_data_within_intervals(b_sakamoto_filtered_results, b_sakamoto_eye_data)
c_watabe_extracted_eye_data = mr.extract_eye_data_within_intervals(c_watabe_filtered_results, c_watabe_eye_data)
d_nakazawa_extracted_eye_data = mr.extract_eye_data_within_intervals(d_nakazawa_filtered_results, d_nakazawa_eye_data)

In [ ]:
def filter_by_variance(extracted_eye_data, filtered_results, threshold):
    filtered_extracted_eye_data = []
    filtered_filtered_results = []

    for i in range(len(extracted_eye_data)):
        diff_x = extracted_eye_data[i]['Gaze point X'].diff().fillna(0)
        diff_y = extracted_eye_data[i]['Gaze point Y'].diff().fillna(0)
        distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離
        variance = distance.var()  # ユークリッド距離の分散を計算

        if variance < threshold:
            filtered_extracted_eye_data.append(extracted_eye_data[i])
            filtered_filtered_results.append(filtered_results[i])

    return filtered_extracted_eye_data, filtered_filtered_results

In [ ]:
Th = 360

In [ ]:
# 分散を計算し、閾値を上回る区間を削除
filtered_extracted_eye_data, filtered_filtered_results = filter_by_variance(extracted_eye_data, filtered_results, Th)

In [ ]:
len(filtered_filtered_results)

In [ ]:
filtered_filtered_results

In [ ]:
a_yuuma_eye_data = mr.process_tobii_csv("datasets/yuuma/20240604/yuuma_eye.csv")

In [ ]:
a_yuuma_eye_data.columns

In [ ]:
a_yuuma_eye_data['Fixation point X']

In [ ]:
# 線形補完を適用
a_yuuma_eye_data['Gaze point X'] = a_yuuma_eye_data['Gaze point X'].interpolate(method='linear')
# 線形補完を適用
a_yuuma_eye_data['Gaze point X'] = a_yuuma_eye_data['Gaze point X'].interpolate(method='linear')

In [ ]:
ylimit = 900

In [ ]:
circle_start_time = pd.Timestamp('2024-06-04 16:00:31')
circle_end_time = pd.Timestamp('2024-06-04 16:00:34')
cross_start_time = pd.Timestamp('2024-06-04 16:10:55')
cross_end_time = pd.Timestamp('2024-06-04 16:10:58')
tri_start_time = pd.Timestamp('2024-06-04 16:31:37')
tri_end_time = pd.Timestamp('2024-06-04 16:31:40')
check_start_time = pd.Timestamp('2024-06-04 16:46:43')
check_end_time = pd.Timestamp('2024-06-04 16:46:46')

# ジェスチャの設定
gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]
# プロットの作成
fig, axes = plt.subplots(4, 1, figsize=(18, 24))  # 4行1列のサブプロットを作成
fig.subplots_adjust(hspace=0.3)  # グラフ間のスペースを広く取る
# 標準偏差を保存するリスト
std_devs = []

for i, (start_time, end_time, label) in enumerate(gestures):
    data = a_yuuma_eye_data[
        (a_yuuma_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (a_yuuma_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    # ハイライトされた区間のデータを抽出
    highlighted_data = data[(data['Timestamp'] >= start_time) & (data['Timestamp'] <= end_time)]
    highlighted_diff_x = highlighted_data['Gaze point X'].interpolate(method='linear').diff()
    highlighted_diff_y = highlighted_data['Gaze point Y'].interpolate(method='linear').diff()
    highlighted_distance = np.sqrt(highlighted_diff_x**2 + highlighted_diff_y**2)

    # 標準偏差の計算
    std_dev = highlighted_distance.std()
    std_devs.append((label, std_dev))

    axes[i].plot(data['Timestamp'], distance, label='Gaze Movement Distance')

    axes[i].grid(True)
    axes[i].set_ylabel('Gaze Movement Distance (pixels)', fontsize=14)
    axes[i].set_xlabel('Timestamp (JST, H:M:S)', fontsize=14)  # 各グラフにx軸ラベルを設定
    axes[i].set_ylim(0, ylimit)  # y軸の範囲をデータに合わせる
    axes[i].set_title(f'{label} Gesture', fontsize=24)
    axes[i].axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Execution Interval')
    axes[i].legend(loc='upper right', fontsize=14)
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  # Timestampフォーマット設定
    experiment_date = start_time.strftime('%Y-%m-%d')
    axes[i].text(0.01, 0.98, f'Experiment Date: {experiment_date}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    # 標準偏差をグラフに表示
    #axes[i].text(0.5, 0.85, f'Std Dev: {std_dev:.2f}', transform=axes[i].transAxes, fontsize=20, verticalalignment='top', horizontalalignment='center')

# プロットの保存（EPS形式）
plt.savefig("/Users/hinase/Downloads/a_yuuma_gesture_Gaze_Movement_Distance.eps", format='eps', bbox_inches='tight')  # 論文用に図を保存

plt.show()  # プロットを表示
# 区間リストの定義
intervals = [
    (a_yuuma_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, a_yuuma_eye_data['Timestamp'].max(), 'Desk work')
]

# 標準偏差を保存するリスト
std_devs = []

# 各区間の標準偏差を計算
for start_time, end_time, label in intervals:
    data = a_yuuma_eye_data[
        (a_yuuma_eye_data['Timestamp'] >= start_time) &
        (a_yuuma_eye_data['Timestamp'] <= end_time)
    ]

    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    std_dev = distance.std()
    std_devs.append((label, std_dev))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Interval', 'Standard Deviation'])
print(std_dev_df)

In [ ]:
circle_start_time = pd.Timestamp('2024-06-04 18:34:55')
circle_end_time = pd.Timestamp('2024-06-04 18:34:59')
cross_start_time = pd.Timestamp('2024-06-04 18:45:10')
cross_end_time = pd.Timestamp('2024-06-04 18:45:14')
tri_start_time = pd.Timestamp('2024-06-04 19:06:48')
tri_end_time = pd.Timestamp('2024-06-04 19:06:52')
check_start_time = pd.Timestamp('2024-06-04 19:21:52')
check_end_time = pd.Timestamp('2024-06-04 19:21:55')

# ジェスチャの設定
gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]
# プロットの作成
fig, axes = plt.subplots(4, 1, figsize=(18, 24))  # 4行1列のサブプロットを作成
fig.subplots_adjust(hspace=0.3)  # グラフ間のスペースを広く取る
# 標準偏差を保存するリスト
std_devs = []

for i, (start_time, end_time, label) in enumerate(gestures):
    data = b_sakamoto_eye_data[
        (b_sakamoto_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (b_sakamoto_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    # ハイライトされた区間のデータを抽出
    highlighted_data = data[(data['Timestamp'] >= start_time) & (data['Timestamp'] <= end_time)]
    highlighted_diff_x = highlighted_data['Gaze point X'].interpolate(method='linear').diff()
    highlighted_diff_y = highlighted_data['Gaze point Y'].interpolate(method='linear').diff()
    highlighted_distance = np.sqrt(highlighted_diff_x**2 + highlighted_diff_y**2)

    # 標準偏差の計算
    std_dev = highlighted_distance.std()
    std_devs.append((label, std_dev))

    axes[i].plot(data['Timestamp'], distance, label='Gaze Movement Distance')

    axes[i].grid(True)
    axes[i].set_ylabel('Gaze Movement Distance (pixels)', fontsize=14)
    axes[i].set_xlabel('Timestamp (JST, H:M:S)', fontsize=14)  # 各グラフにx軸ラベルを設定
    axes[i].set_ylim(0, ylimit)  # y軸の範囲をデータに合わせる
    axes[i].set_title(f'{label} Gesture', fontsize=24)
    axes[i].axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Execution Interval')
    axes[i].legend(loc='upper right', fontsize=14)
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  # Timestampフォーマット設定
    experiment_date = start_time.strftime('%Y-%m-%d')
    axes[i].text(0.01, 0.98, f'Experiment Date: {experiment_date}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    # 標準偏差をグラフに表示
    # axes[i].text(0.99, 0.98, f'Std Dev: {std_dev:.2f}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top', horizontalalignment='right')

# プロットの保存（EPS形式）
plt.savefig("/Users/hinase/Downloads/b_sakamoto_gesture_Gaze_Movement_Distance.eps", format='eps', bbox_inches='tight')  # 論文用に図を保存

plt.show()  # プロットを表示
# 区間リストの定義
intervals = [
    (b_sakamoto_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, b_sakamoto_eye_data['Timestamp'].max(), 'Desk work')
]

# 標準偏差を保存するリスト
std_devs = []

# 各区間の標準偏差を計算
for start_time, end_time, label in intervals:
    data = b_sakamoto_eye_data[
        (b_sakamoto_eye_data['Timestamp'] >= start_time) &
        (b_sakamoto_eye_data['Timestamp'] <= end_time)
    ]

    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    std_dev = distance.std()
    std_devs.append((label, std_dev))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Interval', 'Standard Deviation'])
print(std_dev_df)

In [ ]:
circle_start_time = pd.Timestamp('2024-06-05 15:36:41')
circle_end_time = pd.Timestamp('2024-06-05 15:36:44')
cross_start_time = pd.Timestamp('2024-06-05 15:46:57')
cross_end_time = pd.Timestamp('2024-06-05 15:47:00')
tri_start_time = pd.Timestamp('2024-06-05 16:03:40')
tri_end_time = pd.Timestamp('2024-06-05 16:03:44')
check_start_time = pd.Timestamp('2024-06-05 16:19:06')
check_end_time = pd.Timestamp('2024-06-05 16:19:09')

# ジェスチャの設定
gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]
# プロットの作成
fig, axes = plt.subplots(4, 1, figsize=(18, 24))  # 4行1列のサブプロットを作成
fig.subplots_adjust(hspace=0.3)  # グラフ間のスペースを広く取る
# 標準偏差を保存するリスト
std_devs = []

for i, (start_time, end_time, label) in enumerate(gestures):
    data = c_watabe_eye_data[
        (c_watabe_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (c_watabe_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    # ハイライトされた区間のデータを抽出
    highlighted_data = data[(data['Timestamp'] >= start_time) & (data['Timestamp'] <= end_time)]
    highlighted_diff_x = highlighted_data['Gaze point X'].interpolate(method='linear').diff()
    highlighted_diff_y = highlighted_data['Gaze point Y'].interpolate(method='linear').diff()
    highlighted_distance = np.sqrt(highlighted_diff_x**2 + highlighted_diff_y**2)

    # 標準偏差の計算
    std_dev = highlighted_distance.std()
    std_devs.append((label, std_dev))

    axes[i].plot(data['Timestamp'], distance, label='Gaze Movement Distance')

    axes[i].grid(True)
    axes[i].set_ylabel('Gaze Movement Distance (pixels)', fontsize=14)
    axes[i].set_xlabel('Timestamp (JST, H:M:S)', fontsize=14)  # 各グラフにx軸ラベルを設定
    axes[i].set_ylim(0, ylimit)  # y軸の範囲をデータに合わせる
    axes[i].set_title(f'{label} Gesture', fontsize=24)
    axes[i].axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Execution Interval')
    axes[i].legend(loc='upper right', fontsize=14)
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  # Timestampフォーマット設定
    experiment_date = start_time.strftime('%Y-%m-%d')
    axes[i].text(0.01, 0.98, f'Experiment Date: {experiment_date}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    # 標準偏差をグラフに表示
    # axes[i].text(0.99, 0.98, f'Std Dev: {std_dev:.2f}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top', horizontalalignment='right')

# プロットの保存（EPS形式）
plt.savefig("/Users/hinase/Downloads/c_watabe_gesture_Gaze_Movement_Distance.eps", format='eps', bbox_inches='tight')  # 論文用に図を保存

plt.show()  # プロットを表示
# 区間リストの定義
intervals = [
    (c_watabe_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, c_watabe_eye_data['Timestamp'].max(), 'Desk work')
]

# 標準偏差を保存するリスト
std_devs = []

# 各区間の標準偏差を計算
for start_time, end_time, label in intervals:
    data = c_watabe_eye_data[
        (c_watabe_eye_data['Timestamp'] >= start_time) &
        (c_watabe_eye_data['Timestamp'] <= end_time)
    ]

    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    std_dev = distance.std()
    std_devs.append((label, std_dev))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Interval', 'Standard Deviation'])
print(std_dev_df)

In [ ]:
circle_start_time = pd.Timestamp('2024-06-06 13:44:14')
circle_end_time = pd.Timestamp('2024-06-06 13:44:17')
cross_start_time = pd.Timestamp('2024-06-06 13:54:44')
cross_end_time = pd.Timestamp('2024-06-06 13:54:47')
tri_start_time = pd.Timestamp('2024-06-06 14:14:51')
tri_end_time = pd.Timestamp('2024-06-06 14:14:55')
check_start_time = pd.Timestamp('2024-6-06 14:30:22')
check_end_time = pd.Timestamp('2024-06-06 14:30:25')

# ジェスチャの設定
gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]
# プロットの作成
fig, axes = plt.subplots(4, 1, figsize=(18, 24))  # 4行1列のサブプロットを作成
fig.subplots_adjust(hspace=0.3)  # グラフ間のスペースを広く取る
# 標準偏差を保存するリスト
std_devs = []

for i, (start_time, end_time, label) in enumerate(gestures):
    data = d_nakazawa_eye_data[
        (d_nakazawa_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (d_nakazawa_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    # ハイライトされた区間のデータを抽出
    highlighted_data = data[(data['Timestamp'] >= start_time) & (data['Timestamp'] <= end_time)]
    highlighted_diff_x = highlighted_data['Gaze point X'].interpolate(method='linear').diff()
    highlighted_diff_y = highlighted_data['Gaze point Y'].interpolate(method='linear').diff()
    highlighted_distance = np.sqrt(highlighted_diff_x**2 + highlighted_diff_y**2)

    # 標準偏差の計算
    std_dev = highlighted_distance.std()
    std_devs.append((label, std_dev))

    axes[i].plot(data['Timestamp'], distance, label='Gaze Movement Distance')

    axes[i].grid(True)
    axes[i].set_ylabel('Gaze Movement Distance (pixels)', fontsize=14)
    axes[i].set_xlabel('Timestamp (JST, H:M:S)', fontsize=14)  # 各グラフにx軸ラベルを設定
    axes[i].set_ylim(0, ylimit)  # y軸の範囲をデータに合わせる
    axes[i].set_title(f'{label} Gesture', fontsize=24)
    axes[i].axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Execution Interval')
    axes[i].legend(loc='upper right', fontsize=14)
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  # Timestampフォーマット設定
    experiment_date = start_time.strftime('%Y-%m-%d')
    axes[i].text(0.01, 0.98, f'Experiment Date: {experiment_date}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    # 標準偏差をグラフに表示
    # axes[i].text(0.99, 0.98, f'Std Dev: {std_dev:.2f}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top', horizontalalignment='right')

# プロットの保存（EPS形式）
plt.savefig("/Users/hinase/Downloads/d_nakazawa_gesture_Gaze_Movement_Distance.eps", format='eps', bbox_inches='tight')  # 論文用に図を保存

plt.show()  # プロットを表示
# 区間リストの定義
intervals = [
    (d_nakazawa_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, d_nakazawa_eye_data['Timestamp'].max(), 'Desk work')
]

# 標準偏差を保存するリスト
std_devs = []

# 各区間の標準偏差を計算
for start_time, end_time, label in intervals:
    data = d_nakazawa_eye_data[
        (d_nakazawa_eye_data['Timestamp'] >= start_time) &
        (d_nakazawa_eye_data['Timestamp'] <= end_time)
    ]

    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    std_dev = distance.std()
    std_devs.append((label, std_dev))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Interval', 'Standard Deviation'])
print(std_dev_df)

In [ ]:
circle_start_time = pd.Timestamp('2024-06-07 15:33:19')
circle_end_time = pd.Timestamp('2024-06-07 15:33:22')
cross_start_time = pd.Timestamp('2024-06-07 15:43:54')
cross_end_time = pd.Timestamp('2024-06-07 15:43:57')
tri_start_time = pd.Timestamp('2024-06-07 16:01:46')
tri_end_time = pd.Timestamp('2024-06-07 16:01:49')
check_start_time = pd.Timestamp('2024-06-07 16:17:07')
check_end_time = pd.Timestamp('2024-06-07 16:17:09')

# ジェスチャの設定
gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]
# プロットの作成
fig, axes = plt.subplots(4, 1, figsize=(18, 24))  # 4行1列のサブプロットを作成
fig.subplots_adjust(hspace=0.3)  # グラフ間のスペースを広く取る
# 標準偏差を保存するリスト
std_devs = []

for i, (start_time, end_time, label) in enumerate(gestures):
    data = e_okeda_eye_data[
        (e_okeda_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (e_okeda_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    # ハイライトされた区間のデータを抽出
    highlighted_data = data[(data['Timestamp'] >= start_time) & (data['Timestamp'] <= end_time)]
    highlighted_diff_x = highlighted_data['Gaze point X'].interpolate(method='linear').diff()
    highlighted_diff_y = highlighted_data['Gaze point Y'].interpolate(method='linear').diff()
    highlighted_distance = np.sqrt(highlighted_diff_x**2 + highlighted_diff_y**2)

    # 標準偏差の計算
    std_dev = highlighted_distance.std()
    std_devs.append((label, std_dev))

    axes[i].plot(data['Timestamp'], distance, label='Gaze Movement Distance')

    axes[i].grid(True)
    axes[i].set_ylabel('Gaze Movement Distance (pixels)', fontsize=14)
    axes[i].set_xlabel('Timestamp (JST, H:M:S)', fontsize=14)  # 各グラフにx軸ラベルを設定
    axes[i].set_ylim(0, ylimit)  # y軸の範囲をデータに合わせる
    axes[i].set_title(f'{label} Gesture', fontsize=24)
    axes[i].axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Execution Interval')
    axes[i].legend(loc='upper right', fontsize=14)
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  # Timestampフォーマット設定
    experiment_date = start_time.strftime('%Y-%m-%d')
    axes[i].text(0.01, 0.98, f'Experiment Date: {experiment_date}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    # 標準偏差をグラフに表示
    # axes[i].text(0.99, 0.98, f'Std Dev: {std_dev:.2f}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top', horizontalalignment='right')

# プロットの保存（EPS形式）
plt.savefig("/Users/hinase/Downloads/e_okeda_gesture Gaze_Movement_Distance.eps", format='eps', bbox_inches='tight')  # 論文用に図を保存

plt.show()  # プロットを表示
# 区間リストの定義
intervals = [
    (e_okeda_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, e_okeda_eye_data['Timestamp'].max(), 'Desk work')
]

# 標準偏差を保存するリスト
std_devs = []

# 各区間の標準偏差を計算
for start_time, end_time, label in intervals:
    data = e_okeda_eye_data[
        (e_okeda_eye_data['Timestamp'] >= start_time) &
        (e_okeda_eye_data['Timestamp'] <= end_time)
    ]

    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    std_dev = distance.std()
    std_devs.append((label, std_dev))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Interval', 'Standard Deviation'])
print(std_dev_df)

In [ ]:
ylimit = 1.0

In [ ]:
circle_start_time = pd.Timestamp('2024-06-04 16:00:31')
circle_end_time = pd.Timestamp('2024-06-04 16:00:34')
cross_start_time = pd.Timestamp('2024-06-04 16:10:55')
cross_end_time = pd.Timestamp('2024-06-04 16:10:58')
tri_start_time = pd.Timestamp('2024-06-04 16:31:37')
tri_end_time = pd.Timestamp('2024-06-04 16:31:40')
check_start_time = pd.Timestamp('2024-06-04 16:46:43')
check_end_time = pd.Timestamp('2024-06-04 16:46:46')

# ジェスチャの設定
gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]
# プロットの作成
fig, axes = plt.subplots(4, 1, figsize=(18, 24))  # 4行1列のサブプロットを作成
fig.subplots_adjust(hspace=0.3)  # グラフ間のスペースを広く取る
# 標準偏差を保存するリスト
std_devs = []

for i, (start_time, end_time, label) in enumerate(gestures):
    data = a_yuuma_eye_data[
        (a_yuuma_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (a_yuuma_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    diff_r = data['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data['Pupil diameter left'].interpolate(method='linear').diff()

    axes[i].plot(data['Timestamp'], diff_r, label='Pupil Diameter Right')
    axes[i].plot(data['Timestamp'], diff_l, label='Pupil Diameter Left')
    axes[i].grid(True)
    axes[i].set_ylabel('Pupil Diameter Change (Millimeters)', fontsize=14)
    axes[i].set_xlabel('Timestamp (JST, H:M:S)', fontsize=14)  # 各グラフにx軸ラベルを設定
    axes[i].set_ylim(-1 * ylimit, ylimit)  # y軸の範囲をデータに合わせる
    #axes[i].set_ylim(3, 5)  # y軸の範囲をデータに合わせる
    axes[i].set_title(f'{label} Gesture', fontsize=24)
    axes[i].axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Execution Interval')
    axes[i].legend(loc='upper right', fontsize=14)
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  # Timestampフォーマット設定
    experiment_date = start_time.strftime('%Y-%m-%d')
    axes[i].text(0.01, 0.98, f'Experiment Date: {experiment_date}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    # 標準偏差をグラフに表示
    #axes[i].text(0.5, 0.85, f'Std Dev: {std_dev:.2f}', transform=axes[i].transAxes, fontsize=20, verticalalignment='top', horizontalalignment='center')

# プロットの保存（EPS形式）
plt.savefig("/Users/hinase/Downloads/a_yuuma_gesture_Pupil_Diameter_Change.eps", format='eps', bbox_inches='tight')  # 論文用に図を保存

plt.show()  # プロットを表示
# 区間リストの定義
intervals = [
    (a_yuuma_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, a_yuuma_eye_data['Timestamp'].max(), 'Desk work')
]

# 標準偏差を保存するリスト
std_devs = []

# 各区間の標準偏差を計算
for start_time, end_time, label in intervals:
    data = a_yuuma_eye_data[
        (a_yuuma_eye_data['Timestamp'] >= start_time) &
        (a_yuuma_eye_data['Timestamp'] <= end_time)
    ]

    diff_r = data['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data['Pupil diameter left'].interpolate(method='linear').diff()

    std_dev_r = diff_r.std()
    std_dev_l = diff_l.std()
    std_devs.append((label, std_dev_r, std_dev_l))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Interval', 'Standard Deviation_r', 'Standard Deviation_l'])
print(std_dev_df)

In [ ]:
circle_start_time = pd.Timestamp('2024-06-04 18:34:55')
circle_end_time = pd.Timestamp('2024-06-04 18:34:59')
cross_start_time = pd.Timestamp('2024-06-04 18:45:10')
cross_end_time = pd.Timestamp('2024-06-04 18:45:14')
tri_start_time = pd.Timestamp('2024-06-04 19:06:48')
tri_end_time = pd.Timestamp('2024-06-04 19:06:52')
check_start_time = pd.Timestamp('2024-06-04 19:21:52')
check_end_time = pd.Timestamp('2024-06-04 19:21:55')


# ジェスチャの設定
gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]
# プロットの作成
fig, axes = plt.subplots(4, 1, figsize=(18, 24))  # 4行1列のサブプロットを作成
fig.subplots_adjust(hspace=0.3)  # グラフ間のスペースを広く取る
# 標準偏差を保存するリスト
std_devs = []

for i, (start_time, end_time, label) in enumerate(gestures):
    data = b_sakamoto_eye_data[
        (b_sakamoto_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (b_sakamoto_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    diff_r = data['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data['Pupil diameter left'].interpolate(method='linear').diff()

    axes[i].plot(data['Timestamp'], diff_r, label='Pupil Diameter Right')
    axes[i].plot(data['Timestamp'], diff_l, label='Pupil Diameter Left')
    axes[i].grid(True)
    axes[i].set_ylabel('Pupil Diameter Change (Millimeters)', fontsize=14)
    axes[i].set_xlabel('Timestamp (JST, H:M:S)', fontsize=14)  # 各グラフにx軸ラベルを設定
    axes[i].set_ylim(-1 * ylimit, ylimit)  # y軸の範囲をデータに合わせる
    #axes[i].set_ylim(3, 5)  # y軸の範囲をデータに合わせる
    axes[i].set_title(f'{label} Gesture', fontsize=24)
    axes[i].axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Execution Interval')
    axes[i].legend(loc='upper right', fontsize=14)
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  # Timestampフォーマット設定
    experiment_date = start_time.strftime('%Y-%m-%d')
    axes[i].text(0.01, 0.98, f'Experiment Date: {experiment_date}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    # 標準偏差をグラフに表示
    #axes[i].text(0.5, 0.85, f'Std Dev: {std_dev:.2f}', transform=axes[i].transAxes, fontsize=20, verticalalignment='top', horizontalalignment='center')

# プロットの保存（EPS形式）
plt.savefig("/Users/hinase/Downloads/b_sakamoto_gesture_Pupil_Diameter_Change.eps", format='eps', bbox_inches='tight')  # 論文用に図を保存

plt.show()  # プロットを表示
# 区間リストの定義
intervals = [
    (b_sakamoto_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, b_sakamoto_eye_data['Timestamp'].max(), 'Desk work')
]

# 標準偏差を保存するリスト
std_devs = []

# 各区間の標準偏差を計算
for start_time, end_time, label in intervals:
    data = b_sakamoto_eye_data[
        (b_sakamoto_eye_data['Timestamp'] >= start_time) &
        (b_sakamoto_eye_data['Timestamp'] <= end_time)
    ]

    diff_r = data['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data['Pupil diameter left'].interpolate(method='linear').diff()

    std_dev_r = diff_r.std()
    std_dev_l = diff_l.std()
    std_devs.append((label, std_dev_r, std_dev_l))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Interval', 'Standard Deviation_r', 'Standard Deviation_l'])
print(std_dev_df)

In [ ]:
circle_start_time = pd.Timestamp('2024-06-05 15:36:41')
circle_end_time = pd.Timestamp('2024-06-05 15:36:44')
cross_start_time = pd.Timestamp('2024-06-05 15:46:57')
cross_end_time = pd.Timestamp('2024-06-05 15:47:00')
tri_start_time = pd.Timestamp('2024-06-05 16:03:40')
tri_end_time = pd.Timestamp('2024-06-05 16:03:44')
check_start_time = pd.Timestamp('2024-06-05 16:19:06')
check_end_time = pd.Timestamp('2024-06-05 16:19:09')



# ジェスチャの設定
gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]
# プロットの作成
fig, axes = plt.subplots(4, 1, figsize=(18, 24))  # 4行1列のサブプロットを作成
fig.subplots_adjust(hspace=0.3)  # グラフ間のスペースを広く取る
# 標準偏差を保存するリスト
std_devs = []

for i, (start_time, end_time, label) in enumerate(gestures):
    data = c_watabe_eye_data[
        (c_watabe_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (c_watabe_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    diff_r = data['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data['Pupil diameter left'].interpolate(method='linear').diff()

    axes[i].plot(data['Timestamp'], diff_r, label='Pupil Diameter Right')
    axes[i].plot(data['Timestamp'], diff_l, label='Pupil Diameter Left')
    axes[i].grid(True)
    axes[i].set_ylabel('Pupil Diameter Change (Millimeters)', fontsize=14)
    axes[i].set_xlabel('Timestamp (JST, H:M:S)', fontsize=14)  # 各グラフにx軸ラベルを設定
    axes[i].set_ylim(-1 * ylimit, ylimit)  # y軸の範囲をデータに合わせる
    #axes[i].set_ylim(3, 5)  # y軸の範囲をデータに合わせる
    axes[i].set_title(f'{label} Gesture', fontsize=24)
    axes[i].axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Execution Interval')
    axes[i].legend(loc='upper right', fontsize=14)
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  # Timestampフォーマット設定
    experiment_date = start_time.strftime('%Y-%m-%d')
    axes[i].text(0.01, 0.98, f'Experiment Date: {experiment_date}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    # 標準偏差をグラフに表示
    #axes[i].text(0.5, 0.85, f'Std Dev: {std_dev:.2f}', transform=axes[i].transAxes, fontsize=20, verticalalignment='top', horizontalalignment='center')

# プロットの保存（EPS形式）
plt.savefig("/Users/hinase/Downloads/c_watabe_gesture_Pupil_Diameter_Change.eps", format='eps', bbox_inches='tight')  # 論文用に図を保存

plt.show()  # プロットを表示
# 区間リストの定義
intervals = [
    (c_watabe_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, c_watabe_eye_data['Timestamp'].max(), 'Desk work')
]

# 標準偏差を保存するリスト
std_devs = []

# 各区間の標準偏差を計算
for start_time, end_time, label in intervals:
    data = c_watabe_eye_data[
        (c_watabe_eye_data['Timestamp'] >= start_time) &
        (c_watabe_eye_data['Timestamp'] <= end_time)
    ]

    diff_r = data['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data['Pupil diameter left'].interpolate(method='linear').diff()

    std_dev_r = diff_r.std()
    std_dev_l = diff_l.std()
    std_devs.append((label, std_dev_r, std_dev_l))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Interval', 'Standard Deviation_r', 'Standard Deviation_l'])
print(std_dev_df)

In [ ]:
circle_start_time = pd.Timestamp('2024-06-06 13:44:14')
circle_end_time = pd.Timestamp('2024-06-06 13:44:17')
cross_start_time = pd.Timestamp('2024-06-06 13:54:44')
cross_end_time = pd.Timestamp('2024-06-06 13:54:47')
tri_start_time = pd.Timestamp('2024-06-06 14:14:51')
tri_end_time = pd.Timestamp('2024-06-06 14:14:55')
check_start_time = pd.Timestamp('2024-6-06 14:30:22')
check_end_time = pd.Timestamp('2024-06-06 14:30:25')




# ジェスチャの設定
gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]
# プロットの作成
fig, axes = plt.subplots(4, 1, figsize=(18, 24))  # 4行1列のサブプロットを作成
fig.subplots_adjust(hspace=0.3)  # グラフ間のスペースを広く取る
# 標準偏差を保存するリスト
std_devs = []

for i, (start_time, end_time, label) in enumerate(gestures):
    data = d_nakazawa_eye_data[
        (d_nakazawa_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (d_nakazawa_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    diff_r = data['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data['Pupil diameter left'].interpolate(method='linear').diff()

    axes[i].plot(data['Timestamp'], diff_r, label='Pupil Diameter Right')
    axes[i].plot(data['Timestamp'], diff_l, label='Pupil Diameter Left')
    axes[i].grid(True)
    axes[i].set_ylabel('Pupil Diameter Change (Millimeters)', fontsize=14)
    axes[i].set_xlabel('Timestamp (JST, H:M:S)', fontsize=14)  # 各グラフにx軸ラベルを設定
    axes[i].set_ylim(-1 * ylimit, ylimit)  # y軸の範囲をデータに合わせる
    #axes[i].set_ylim(3, 5)  # y軸の範囲をデータに合わせる
    axes[i].set_title(f'{label} Gesture', fontsize=24)
    axes[i].axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Execution Interval')
    axes[i].legend(loc='upper right', fontsize=14)
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  # Timestampフォーマット設定
    experiment_date = start_time.strftime('%Y-%m-%d')
    axes[i].text(0.01, 0.98, f'Experiment Date: {experiment_date}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    # 標準偏差をグラフに表示
    #axes[i].text(0.5, 0.85, f'Std Dev: {std_dev:.2f}', transform=axes[i].transAxes, fontsize=20, verticalalignment='top', horizontalalignment='center')

# プロットの保存（EPS形式）
plt.savefig("/Users/hinase/Downloads/d_nakazawa_gesture_Pupil_Diameter_Change.eps", format='eps', bbox_inches='tight')  # 論文用に図を保存

plt.show()  # プロットを表示
# 区間リストの定義
intervals = [
    (d_nakazawa_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, d_nakazawa_eye_data['Timestamp'].max(), 'Desk work')
]

# 標準偏差を保存するリスト
std_devs = []

# 各区間の標準偏差を計算
for start_time, end_time, label in intervals:
    data = d_nakazawa_eye_data[
        (d_nakazawa_eye_data['Timestamp'] >= start_time) &
        (d_nakazawa_eye_data['Timestamp'] <= end_time)
    ]

    diff_r = data['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data['Pupil diameter left'].interpolate(method='linear').diff()

    std_dev_r = diff_r.std()
    std_dev_l = diff_l.std()
    std_devs.append((label, std_dev_r, std_dev_l))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Interval', 'Standard Deviation_r', 'Standard Deviation_l'])
print(std_dev_df)

In [ ]:
circle_start_time = pd.Timestamp('2024-06-07 15:33:19')
circle_end_time = pd.Timestamp('2024-06-07 15:33:22')
cross_start_time = pd.Timestamp('2024-06-07 15:43:54')
cross_end_time = pd.Timestamp('2024-06-07 15:43:57')
tri_start_time = pd.Timestamp('2024-06-07 16:01:46')
tri_end_time = pd.Timestamp('2024-06-07 16:01:49')
check_start_time = pd.Timestamp('2024-06-07 16:17:07')
check_end_time = pd.Timestamp('2024-06-07 16:17:09')



# ジェスチャの設定
gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]
# プロットの作成
fig, axes = plt.subplots(4, 1, figsize=(18, 24))  # 4行1列のサブプロットを作成
fig.subplots_adjust(hspace=0.3)  # グラフ間のスペースを広く取る
# 標準偏差を保存するリスト
std_devs = []

for i, (start_time, end_time, label) in enumerate(gestures):
    data = e_okeda_eye_data[
        (e_okeda_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (e_okeda_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    diff_r = data['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data['Pupil diameter left'].interpolate(method='linear').diff()

    axes[i].plot(data['Timestamp'], diff_r, label='Pupil Diameter Right')
    axes[i].plot(data['Timestamp'], diff_l, label='Pupil Diameter Left')
    axes[i].grid(True)
    axes[i].set_ylabel('Pupil Diameter Change (Millimeters)', fontsize=14)
    axes[i].set_xlabel('Timestamp (JST, H:M:S)', fontsize=14)  # 各グラフにx軸ラベルを設定
    axes[i].set_ylim(-1 * ylimit, ylimit)  # y軸の範囲をデータに合わせる
    #axes[i].set_ylim(3, 5)  # y軸の範囲をデータに合わせる
    axes[i].set_title(f'{label} Gesture', fontsize=24)
    axes[i].axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Execution Interval')
    axes[i].legend(loc='upper right', fontsize=14)
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  # Timestampフォーマット設定
    experiment_date = start_time.strftime('%Y-%m-%d')
    axes[i].text(0.01, 0.98, f'Experiment Date: {experiment_date}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    # 標準偏差をグラフに表示
    #axes[i].text(0.5, 0.85, f'Std Dev: {std_dev:.2f}', transform=axes[i].transAxes, fontsize=20, verticalalignment='top', horizontalalignment='center')

# プロットの保存（EPS形式）
plt.savefig("/Users/hinase/Downloads/e_okeda_gesture_Pupil_Diameter_Change.eps", format='eps', bbox_inches='tight')  # 論文用に図を保存

plt.show()  # プロットを表示
# 区間リストの定義
intervals = [
    (e_okeda_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, e_okeda_eye_data['Timestamp'].max(), 'Desk work')
]

# 標準偏差を保存するリスト
std_devs = []

# 各区間の標準偏差を計算
for start_time, end_time, label in intervals:
    data = e_okeda_eye_data[
        (e_okeda_eye_data['Timestamp'] >= start_time) &
        (e_okeda_eye_data['Timestamp'] <= end_time)
    ]

    diff_r = data['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data['Pupil diameter left'].interpolate(method='linear').diff()

    std_dev_r = diff_r.std()
    std_dev_l = diff_l.std()
    std_devs.append((label, std_dev_r, std_dev_l))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Interval', 'Standard Deviation_r', 'Standard Deviation_l'])
print(std_dev_df)